In [1]:
import pandas as pd
train  = pd.read_csv("train_dataset_train.csv")
test = pd.read_csv("test_dataset_test.csv")

train.loc[train["Год_Поступления"] == 2212, "Год_Поступления"] = 2012


In [4]:
(train["Статус"] == -1).sum()

614

In [18]:
def fix_sex(df):
    df.loc[df["Пол"] == "муж", "Пол"] = "Муж"
    df.loc[df["Пол"] == "жен", "Пол"] = "Жен"

    df.loc[df["Пол"].isna(), "Пол"] = "None"
    return df

In [19]:
def fix_language(df):
    df["Изучаемый_Язык"] = df["Изучаемый_Язык"].fillna("None")
    
    df.loc[df["Изучаемый_Язык"].apply(lambda x: False if(x is None) else ("Английский" in x)), "Изучаемый_Язык"] = "Английский"
    df.loc[df["Изучаемый_Язык"].apply(lambda x: False if(x is None) else ("Англиийский" in x)), "Изучаемый_Язык"] = "Английский"
    df.loc[df["Изучаемый_Язык"].apply(lambda x: False if(x is None) else ("Немецкий" in x)), "Изучаемый_Язык"] = "Немецкий"
    df.loc[df["Изучаемый_Язык"].apply(lambda x: False if(x is None) else ("Французский" in x)), "Изучаемый_Язык"] = "Французский"
    
    return df


In [20]:
def parse_date(x):
    year = int(x.split('-')[0])
    month = x.split('-')[1]
    if(month[0]=="0"):
        month = month[1]
        
    month =  int(month)
    
    return year*12 + month

def parse_post(x):
    return x*12 + 9
    
def fix_dates(df):
    df["Дата_Рождения"] = df["Дата_Рождения"].apply(parse_date)
    df["Год_Окончания_УЗ"] = df["Год_Окончания_УЗ"].fillna(-1)
    df["gape"] = df["Год_Поступления"].values - df["Год_Окончания_УЗ"].values
    df["gape_age"] = df["Год_Поступления"].fillna(-1).apply(parse_post).values - df["Дата_Рождения"].values
    
    return df


In [21]:
#this set of features is aimed on seeking differences between peoples who relocated in order to study in university
#and those who were not

def find_lcsubstr(s1, s2):
    m = [[0 for i in range(len(s2) + 1)] for j in range(len(s1) + 1)]  # Сгенерировать матрицу 0, чтобы облегчить последующие вычисления, на один столбец больше, чем длина строки
    mmax = 0  # Длина самого длинного совпадения
    p = 0  # Самое длинное совпадение соответствует последнему биту в s1
    for i in range(len(s1)):
        for j in range(len(s2)):
            if s1[i] == s2[j]: # Если равно, добавить существующую общую подстроку
                m[i + 1][j + 1] = m[i][j] + 1
                if m[i + 1][j + 1] > mmax:
                    mmax = m[i + 1][j + 1]
                    p = i + 1
    return s1[p - mmax:p], mmax

def add_territorial_names_simmilarity(df):
    names = ["Страна_ПП", "Страна_Родители", "Город_ПП", "Регион_ПП", "Где_Находится_УЗ", "Уч_Заведение"]

    for i in range(len(names)-1):
        for j in range(i + 1, len(names)):
            v = []
            for x, y in zip(df[names[i]], df[names[j]]):
                if pd.notnull(x) and pd.notnull(y):
                    #print(i, j, x,y)
                    val = find_lcsubstr(x, y)[1]/min(len(x), len(y))
                    v.append(val)
                else:
                    v.append(-1)

        df[names[i] + '_' + names[j] + "_diff"] = v
    return df



In [22]:
def cityf(st):
    r = ''
    if pd.notnull(st):
    
        for w in st.split(' '):
            if(len(w)==0):
                continue
            if(w[0].isupper()):
                r += w
        
    if(len(r) == 0):
        r = 'none'
    return r

#many regions have different ways of writing. So I unify them by word roots.
def fix_territorial_names(df):
    df["Страна_ПП"] = df["Страна_ПП"].fillna("None")
    
    df["Страна_ПП"] = df["Страна_ПП"].apply(lambda x: x.lower())
    
    df.loc[train["Страна_ПП"].apply(lambda x: ("казах" in x)), "Страна_ПП"] = "казах"
    df.loc[train["Страна_ПП"].apply(lambda x: ("таджик" in x)), "Страна_ПП"] = "таджик"
    df.loc[train["Страна_ПП"].apply(lambda x: ("кырг" in x)or("кирг" in x)), "Страна_ПП"] = "кырг"
    df.loc[train["Страна_ПП"].apply(lambda x: ("росс" in x)), "Страна_ПП"] = "росс"
    
    df["Страна_Родители"] = train["Страна_Родители"].fillna("None")
    df["Страна_Родители"] = train["Страна_Родители"].apply(lambda x: x.lower())

    df.loc[train["Страна_Родители"].apply(lambda x: ("казах" in x)), "Страна_Родители"] = "казах"
    df.loc[train["Страна_Родители"].apply(lambda x: ("таджик" in x)), "Страна_Родители"] = "таджик"
    df.loc[train["Страна_Родители"].apply(lambda x: ("кырг" in x)or("кирг" in x)), "Страна_Родители"] = "кырг"
    df.loc[train["Страна_Родители"].apply(lambda x: ("росс" in x)), "Страна_Родители"] = "росс"
    df.loc[train["Страна_Родители"].apply(lambda x: ("кнр" in x)), "Страна_Родители"] = "китай"
    #city
    df["Город_ПП"] = df["Город_ПП"].apply(cityf)
    #---
    df["Регион_ПП"] = df["Регион_ПП"].fillna("none")
    df["Регион_ПП"] = df["Регион_ПП"].apply(lambda a: a.lower())
    
    df["reg_pp"] = df["Регион_ПП"].copy()
    df["reg_pp"] = df["reg_pp"].fillna("none")

    
    ss = set([ "алтайс",
     "казах",
     "тыва",
     "кемер", 
      "саха",
      "новосиб",
      "павлодар",
     "алтайс",
     "краснояр",
      "жалал",
     "хатлон",
     "казах",
     "алмат",
     "ляонин",
     "хайнань",
     "хэйлун",
     "ямало",
      "томск",
      "цзилинь",
      "иркут",
      "бадах",
      "иркут",
      "ханты"])


    df.loc[df["Регион_ПП"].apply(lambda x: ("алтай" in x)), "Регион_ПП"] = "алтайс"
    df.loc[df["Регион_ПП"].apply(lambda x: ("казах" in x)), "Регион_ПП"] = "казах"
    df.loc[df["Регион_ПП"].apply(lambda x: ("тыва" in x)), "Регион_ПП"] = "тыва"
    df.loc[df["Регион_ПП"].apply(lambda x: ("кемер" in x)), "Регион_ПП"] = "кемер"
    df.loc[df["Регион_ПП"].apply(lambda x: ("саха" in x)), "Регион_ПП"] = "саха"
    df.loc[df["Регион_ПП"].apply(lambda x: ("новосиб" in x)), "Регион_ПП"] = "новосиб"
    df.loc[df["Регион_ПП"].apply(lambda x: ("павлодар" in x)), "Регион_ПП"] = "павлодар"
    df.loc[df["Регион_ПП"].apply(lambda x: ("алай" in x)), "Регион_ПП"] = "алтайс"
    df.loc[df["Регион_ПП"].apply(lambda x: ("краснояр" in x)), "Регион_ПП"] = "краснояр"
    df.loc[df["Регион_ПП"].apply(lambda x: ("жалал" in x)), "Регион_ПП"] = "жалал"
    df.loc[df["Регион_ПП"].apply(lambda x: ("хатлон" in x)), "Регион_ПП"] = "хатлон"
    df.loc[df["Регион_ПП"].apply(lambda x: ("вко" in x)), "Регион_ПП"] = "казах"
    df.loc[df["Регион_ПП"].apply(lambda x: ("алмат" in x)), "Регион_ПП"] = "алмат"
    df.loc[df["Регион_ПП"].apply(lambda x: ("ляонин" in x)), "Регион_ПП"] = "ляонин"
    df.loc[df["Регион_ПП"].apply(lambda x: ("хайнань" in x)), "Регион_ПП"] = "хайнань"
    df.loc[df["Регион_ПП"].apply(lambda x: ("хэйлун" in x)), "Регион_ПП"] = "хэйлун"
    df.loc[df["Регион_ПП"].apply(lambda x: ("ямало" in x)), "Регион_ПП"] = "ямало"
    df.loc[df["Регион_ПП"].apply(lambda x: ("томск" in x)), "Регион_ПП"] = "томск"
    df.loc[df["Регион_ПП"].apply(lambda x: ("цзилинь" in x)), "Регион_ПП"] = "цзилинь"
    df.loc[df["Регион_ПП"].apply(lambda x: ("иркут" in x)), "Регион_ПП"] = "иркут"
    df.loc[df["Регион_ПП"].apply(lambda x: ("бадах" in x)), "Регион_ПП"] = "бадах"
    df.loc[df["Регион_ПП"].apply(lambda x: ("иркут" in x)), "Регион_ПП"] = "иркут"
    df.loc[df["Регион_ПП"].apply(lambda x: ("ханты" in x)), "Регион_ПП"] = "ханты"
    df.loc[df["Регион_ПП"].apply(lambda x: ("омск" in x)), "Регион_ПП"] = "омск"

    df.loc[~df["Регион_ПП"].isin(ss), "Регион_ПП"] = "rare"
    #-----------
    df["Где_Находится_УЗ"] = df["Где_Находится_УЗ"].fillna("none")
    df["Где_Находится_УЗ"] = df["Где_Находится_УЗ"].apply(lambda a:a.split(",")[-1]).apply(cityf)
    #-----------
    
    return df



In [23]:
#dataset contain certain amount of boolean variavles. I transform them into categorical with 3 values 0, 1, none==-1
#I found out that model benefits from such NaN fillings strategy
def fix_flags(df):
    df["Пособие"] = df["Пособие"].fillna(-1)
    
    df["Общежитие"] = df["Общежитие"].fillna(-1)
    
    df["Наличие_Матери"] = df["Наличие_Матери"].fillna(-1)
    
    df["Наличие_Отца"] = df["Наличие_Отца"].fillna(-1)
    
    df["Иностранец"] = df["Иностранец"].fillna(-1)
    
    df["Опекунство"] = df["Опекунство"].fillna(-1)
    
    df["Село"] = df["Село"].fillna(-1)
    
    return df
    

In [24]:
from collections import Counter
import numpy as np
#function which exctracts bag of words features
def transform_place(st, words2id):
    x = np.zeros((len(words2id), ))
    
    if pd.notnull(st):
        st = st.lower()
        rez = ''.join([x if(x.isalpha() or x.isdigit()) else ' ' for x in st]).split()
        for w in rez:
            if w in words2id:
                x[words2id[w]] += 1
    return x

In [25]:
#this variables will be signed as categorical for LGBM
cat_feat = ["Пол", "Основания", "Изучаемый_Язык", "Пособие", 
            "Страна_ПП", "Общежитие", "Наличие_Матери", "Наличие_Отца", "Страна_Родители", "Опекунство", 
            "Село", "Иностранец", "КодФакультета", "Город_ПП", "Регион_ПП", "Где_Находится_УЗ", "Уч_Заведение", "reg_pp"]

#drop answers from dataset))
drop_feat = [ "Статус"] # 'ID', "Код_группы",

In [26]:
import numpy as np

def merge_common(df, work_count, work_diff, work_diff_inv, dfid, dfidinv, uchzavdf):
    #extract extra fetures from row with dataleak
    df["f1"] = df["Код_группы"]//10000
    df["f2"] = df["Код_группы"]//1000
    df["f3"] = df["Код_группы"]//100
    df["f4"] = df["Код_группы"]//10

    df["l1"] = df["Код_группы"]%10000
    df["l2"] = df["Код_группы"]%1000
    df["l3"] = df["Код_группы"]%100
    df["l4"] = df["Код_группы"]%10

    df["df"] = df["Код_группы"]%2000
    df["dl"] = df["Код_группы"]//2000
    
    count_name = "Код_группы_count"
    name = "Код_группы"
    
    
    #merge statistics which was calculated from train and test simultaneously

    df = df.merge(work_count, on=name, how="left") 
    df = df.merge(work_diff, on=name, how="left") 
    df = df.merge(work_diff_inv, on=name, how="left") 
    df = df.merge(dfid, on="ID", how="left") 
    df = df.merge(dfidinv, on="ID", how="left") 


    df = pd.concat([df, uchzavdf], axis=1)
    

    return df

def common_calculated_features(train, test, cat_feat, drop_feat):
    
    for fixer in [fix_sex, fix_language, fix_dates, add_territorial_names_simmilarity, fix_territorial_names, fix_flags]:
        train = fixer(train)
        test = fixer(test)
        
    
    #here I'm trying to squeze extra information from "leaked" rows

    df = pd.concat([train, test], axis=0)
    count_name = "Код_группы_count"
    name = "Код_группы"
    work_count = df[[name]].copy()
    work_count[count_name] = 1
    work_count = work_count.groupby(name).sum().reset_index()

    sg = set(df[df["СрБаллАттестата"]>5]["Код_группы"])&set(df[df["СрБаллАттестата"]<=5]["Код_группы"])
    a = np.zeros((len(train), ))
    a[train["Код_группы"].isin(sg)] = 1
    train["Strange_disp"] = a

    a = np.zeros((len(test), ))
    a[test["Код_группы"].isin(sg)] = 1
    test["Strange_disp"] = a

    a = np.array(list(set(df["Код_группы"].values)))
    a.sort()
    b = np.hstack([np.array([-1]), a[1:] - a[:-1]])
    work_diff = pd.DataFrame.from_dict({"Код_группы":a, "diff_group":b})

    a = np.array(list(set(df["Код_группы"].values)))
    a.sort()
    b = np.hstack([a[1:] - a[:-1], np.array([-1])])
    work_diff_inv = pd.DataFrame.from_dict({"Код_группы":a, "diff_group_inv":b})

    a = np.array(list(set(df["ID"].values)))
    a.sort()
    b = np.hstack([np.array([-1]), a[1:] - a[:-1]])
    dfid = pd.DataFrame.from_dict({"ID":a, "diff_ID":b})

    a = np.array(list(set(df["ID"].values)))
    a.sort()
    b = np.hstack([a[1:] - a[:-1], np.array([-1])])
    dfidinv = pd.DataFrame.from_dict({"ID":a, "diff_ID_ind":b})
    
    #extract bow features from previous place of study
    ct = Counter()
    for st in train[~train["Уч_Заведение"].isna()]["Уч_Заведение"]:
        st = st.lower()
        rez = ''.join([x if(x.isalpha()or x.isdigit()) else ' ' for x in st])
        ct += Counter( rez.split() ) 

    filtered_words = set([word for word, freq in ct.most_common() if freq>20])
    words2id  = {word:i for i, word in enumerate(filtered_words)}
    id2word = {v:k for k,v in words2id.items()}
    
    A = np.vstack(train["Уч_Заведение"].apply(lambda x: transform_place(x, words2id)).tolist())
    uchzavdf = pd.DataFrame(A, columns = [id2word[i]+"_учзав" for i in range(len(filtered_words))])

    A = np.vstack(test["Уч_Заведение"].apply(lambda x: transform_place(x, words2id)).tolist())
    uchzavdf_test = pd.DataFrame(A, columns = [id2word[i]+"_учзав" for i in range(len(filtered_words))])
    
    #mark categorical features
    feature_columns = list(train.columns.values)
    for x in drop_feat:
        feature_columns.remove(x)
    
    X_TRAIN = train[feature_columns].copy()
    X_TEST = test[feature_columns].copy()
    y = train["Статус"].values
    
    for x in cat_feat:
        X_TRAIN[x] = X_TRAIN[x].astype("category")
        X_TEST[x] = X_TEST[x].astype("category")

        

    X_TRAIN = merge_common(X_TRAIN, work_count, work_diff, work_diff_inv, dfid, dfidinv, uchzavdf)
    X_TEST = merge_common(X_TEST, work_count, work_diff, work_diff_inv, dfid, dfidinv, uchzavdf_test)
    
    
    X_TRAIN["Status"] = y
    X_TRAIN = X_TRAIN.sort_values(["Код_группы", "ID"])
    y = X_TRAIN["Status"].values
    X_TRAIN = X_TRAIN.drop(columns=["Status"])
    X_TRAIN = X_TRAIN.reset_index()
    X_TRAIN = X_TRAIN.drop(columns=["index"])
    
    
    return X_TRAIN, y, X_TEST 

In [27]:
#there should have been several layer of stacking.
#but I find out that it is not benefical for final solution
#So that functions may look little bit sophisticated
def most_common(lst):
    return max(set(lst), key=lst.count)

def fit_lgbms(params, cat_feat, X, y):
    lgbms = []
    seeds = [154, 2277, 42, 5, 8713]
    for seed in seeds:
        
        lgbm = LGBMClassifier(is_unbalance=True, **params, random_state=seed)
        
        lgbm.fit(X, y, categorical_feature=cat_feat)
        
        lgbms.append(lgbm)
        
    return lgbms

def fit_lgbms_full(params, cat_feat, X, y):
    lgbms = []
    seeds = [154, 2277, 42, 5, 8713]
    for seed in seeds:
        
        lgbm = LGBMClassifier(class_weight="balanced", **params, random_state=seed)
        
        lgbm.fit(X, y, categorical_feature=cat_feat)
        
        lgbms.append(lgbm)
        
    return lgbms


def prdict_lgbms(lgbms, X):
    
    ys = []
    
    for lgbm in lgbms:
        
        y = lgbm.predict(X)
        ys.append(y)
        
    yf = []
    for i in range(len(X)):
        l = [ys[j][i] for j in range(len(lgbms))]
        yf.append(most_common(l))
           
    return yf


def prdict_lgbms_proba(lgbms, X):
    
    ys = []
    
    for lgbm in lgbms:
        
        y = lgbm.predict_proba(X)[:,1].reshape(1, -1)
        ys.append(y)
               
    return np.vstack(ys).mean(axis=0)

def prdict_lgbms_proba_full(lgbms, X):
    
    ys = []
    
    for lgbm in lgbms:
        
        y = lgbm.predict_proba(X) #[:,1].reshape(1, -1)
        ys.append(y)
               
    return np.stack(ys).mean(axis=0)

In [28]:
def sm_bagged_fit_predict(X_TRAIN, y, X_TEST, sm_params, n_folds, seeds):
    ovo_test_list = []
    sm_train_list = []
       
    features_sm_test = []
        
    for seed in seeds:
        
        features_ovo = []

        preds = []
    

        for ti, di in StratifiedKFold(n_splits=n_folds, shuffle=True,  random_state=seed).split(X_TRAIN, y):

            Xt = X_TRAIN.iloc[ti]
            yt = y[ti]


            Xd = X_TRAIN.iloc[di]
            yd = y[di]

            lgbms = fit_lgbms_full(sm_params, cat_feat, Xt, yt)
            ypred = prdict_lgbms_proba_full(lgbms, Xd)
            features_sm_test.append(prdict_lgbms_proba_full(lgbms, X_TEST))

            preds.append((di, ypred))

                    
        
        yprom = np.zeros((len(X_TRAIN), 3))

        for i in range(n_folds):
            yprom[preds[i][0]] = preds[i][1]

        sm_train_list.append(yprom)
    
    return np.stack(features_sm_test).mean(axis=0), np.stack(sm_train_list).mean(axis=0)

In [29]:
#optuna selected hyperparametrs
sm_params = {'n_estimators': 500,
 'learning_rate': 0.01587687545192069,
 'num_leaves': 260,
 'min_child_samples': 65,
 'min_split_gain': 0.00044563404841053283,
 'subsample': 0.9,
 'subsample_freq': 1,
 'colsample_bytree': 0.4,
 'max_cat_threshold': 8,
 'cat_l2': 94.8926686487861,
 'min_sum_hessian_in_leaf': 0.090931378259608,
 'max_depth': -1}

In [ ]:
X_TRAIN, y, X_TEST = common_calculated_features(train, test, cat_feat, drop_feat)

In [ ]:
X, Ct = sm_bagged_fit_predict(X_TRAIN, y, X_TEST, sm_params, 5, [333, 444, 555, 777, 888]) #[333, 444, 555, 777, 888]
Cd = X

In [ ]:
ypred = np.array([-1, 3, 4])[np.argmax(Cd, axis=1)] #Cd

subm = pd.read_csv("sample_submission.csv")
subm["Статус"] = ypred
subm.to_csv("rez.csv", encoding='utf-8', sep=',', index=False)